In [1]:
%cd '/content/drive/MyDrive/Final_Sem_Project'

/content/drive/MyDrive/Final_Sem_Project


In [2]:
import os
import random
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [3]:
def seed_everything(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
seed_everything()

In [4]:
data = pd.read_csv("data.csv")

data = shuffle(data)

data.columns = ['height', 'radius', 'freq', 'S11']

Y = data['S11']
X = data.drop('S11', axis = 1)

print(X.shape)
print(Y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 42, shuffle = True)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

def printLosses(y_pred, y_test):
  R2_SCORE = r2_score(y_test, y_pred)
  MSE = mean_squared_error(y_test, y_pred)
  MAE = mean_absolute_error(y_test, y_pred)
  RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
  MAPE = mean_absolute_percentage_error(y_test, y_pred)
  RMSPE = (np.sqrt(np.mean(np.square((y_test - y_pred) / y_test)))) #* 100

  print("R2 Score on test set is : ", R2_SCORE)
  print("Mean Squared Error on test set is : ", MSE)
  print("Mean Absolute Error on test set is : ", MAE)
  print("Root Mean Squared Error on test set is : ", RMSE)
  print("Mean Absolute Percentage Error on test set is : ", MAPE)
  print("Root Mean Squared Percentage Error on test set is : ", RMSPE)

(48400, 3)
(48400,)
(36300, 3)
(36300,)
(12100, 3)
(12100,)


In [5]:
from sklearn.neighbors import KNeighborsRegressor

knn3 = KNeighborsRegressor(n_neighbors = 3)
knn3.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=3)

In [6]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 700)
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=700)

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.8.0


In [8]:
dnn_model = keras.models.load_model("keras_model_1")

In [9]:
dnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normalizat  (None, 3)                7         
 ion)                                                            
                                                                 
 dense (Dense)               (None, 32)                128       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 6,472
Trainable params: 6,465
Non-trainable params: 7
______________________________________________________

In [10]:
test_dataset = pd.read_csv('test_dataset.csv')

In [11]:
test_dataset.head()

,Freq [GHz],"dB(S(1,1)) []"
0,4.000000,-4.172195
1,4.020050,-4.403304
2,4.040100,-4.644045
3,4.060150,-4.894359
4,4.080201,-5.154112


In [12]:
# Height - 10mm
# Radius - 8mm

In [13]:
# test_dataset['Freq [GHz]']

In [14]:
test_data = pd.DataFrame(columns = ['height', 'radius', 'freq'])

In [15]:
test_data['freq'] = test_dataset['Freq [GHz]']

In [16]:
test_data['height'] = 10

In [17]:
test_data['radius'] = 8

In [18]:
test_data

,height,radius,freq
0,10,8,4.000000
1,10,8,4.020050
2,10,8,4.040100
3,10,8,4.060150
4,10,8,4.080201
...,...,...,...
395,10,8,11.919799
396,10,8,11.939850
397,10,8,11.959900
398,10,8,11.979950


In [19]:
knn_s11 = knn3.predict(test_data)

In [20]:
rf_s11 = rf.predict(test_data)

In [21]:
y_pred = dnn_model.predict(test_data)
dnn_s11 = np.ravel(y_pred)

In [22]:
HFSS_s11 = test_dataset['dB(S(1,1)) []']

In [23]:
S11_Freq = pd.DataFrame(columns = ['freq', 'HFSS', 'KNN(N=3)', 'Random Forest', 'DNN'])

In [24]:
S11_Freq['freq'] = test_dataset['Freq [GHz]']
S11_Freq['HFSS'] = test_dataset['dB(S(1,1)) []']
S11_Freq['KNN(N=3)'] = knn_s11
S11_Freq['Random Forest'] = rf_s11
S11_Freq['DNN'] = dnn_s11

In [25]:
df2 = S11_Freq[(S11_Freq.freq >= 7) & (S11_Freq.freq <= 11)]

In [26]:
df2

,freq,HFSS,KNN(N=3),Random Forest,DNN
150,7.007519,-2.598010,-2.598498,-2.599722,-2.565742
151,7.027569,-2.612699,-2.613251,-2.615807,-2.588032
152,7.047619,-2.629046,-2.629671,-2.631988,-2.610324
153,7.067669,-2.647268,-2.647977,-2.652688,-2.632616
154,7.087719,-2.667617,-2.668421,-2.663439,-2.654906
...,...,...,...,...,...
345,10.917293,-4.897967,-4.697049,-4.861013,-4.941724
346,10.937343,-4.575789,-4.697049,-4.515759,-4.658062
347,10.957393,-4.270688,-4.178922,-4.253557,-4.447852
348,10.977444,-3.976990,-3.789362,-4.133554,-4.291023


In [27]:
import plotly.express as px

In [28]:
fig = px.line(df2, x = 'freq', y = ['HFSS', 'KNN(N=3)', 'Random Forest', 'DNN'], width=700, height=400)
fig.update_layout(
    title='S11 vs Freq Comparison Graph',
    title_x=0.45,
    xaxis_title='Freq [GHz]',
    yaxis_title='S11 [dB]'
)
fig.show()

In [29]:
df2.columns

Index(['freq', 'HFSS', 'KNN(N=3)', 'Random Forest', 'DNN'], dtype='object')

In [30]:
printLosses(df2['HFSS'], df2['KNN(N=3)'])

R2 Score on test set is :  0.9594552718887491
Mean Squared Error on test set is :  0.6662552268581534
Mean Absolute Error on test set is :  0.30929953901246104
Root Mean Squared Error on test set is :  0.8162445876440183
Mean Absolute Percentage Error on test set is :  0.03636467863490639
Root Mean Squared Percentage Error on test set is :  0.06806359300525607


In [31]:
printLosses(df2['HFSS'], df2['Random Forest'])

R2 Score on test set is :  0.9595016285756391
Mean Squared Error on test set is :  0.6469608965694762
Mean Absolute Error on test set is :  0.31766864633234415
Root Mean Squared Error on test set is :  0.8043387946440705
Mean Absolute Percentage Error on test set is :  0.0398531749219967
Root Mean Squared Percentage Error on test set is :  0.0744923968527547


In [32]:
printLosses(df2['HFSS'], df2['DNN'])

R2 Score on test set is :  0.9676779359568628
Mean Squared Error on test set is :  0.5568286077461486
Mean Absolute Error on test set is :  0.4007357565775484
Root Mean Squared Error on test set is :  0.7462094932029132
Mean Absolute Percentage Error on test set is :  0.054142754419165245
Root Mean Squared Percentage Error on test set is :  0.07537751535701982


In [33]:
print("R2 Score : ", np.mean([0.967, 0.959, 0.959]))

R2 Score :  0.9616666666666666


In [34]:
print("MSE : ", np.mean([0.556, 0.646, 0.666]))

MSE :  0.6226666666666666


In [35]:
print("MAE : ", np.mean([0.400, 0.317, 0.309]))

MAE :  0.342
